In [1]:
# !aws s3 sync s3://zillow-kaggle-awsml-2020 .
!pip install tqdm

In [2]:
import pandas as pd
df_2017 = pd.read_csv("train_2017.csv")
df_2016 = pd.read_csv("train_2016_v2.csv")

In [3]:
df_properties_2017 = pd.read_csv("properties_2017.csv")
df_properties_2016 = pd.read_csv("properties_2016.csv")

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_2017_joined = df_2017.merge(df_properties_2017, on=['parcelid'],how='left')

In [5]:
df_2016_joined = df_2016.merge(df_properties_2016, on=['parcelid'],how='left')

In [6]:
df_all_joined = pd.concat([df_2017_joined, df_2016_joined])

In [7]:
import numpy as np
import tqdm
from tqdm import tqdm
tqdm.pandas()
features_to_transform = ['fireplacecnt','fullbathcnt','garagecarcnt','calculatedbathnbr','bedroomcnt','bathroomcnt','finishedfloor1squarefeet','garagetotalsqft']

def transform_feature(x):
    if np.isnan(x):
        return 0
    else:
        log_res = np.log10(x + 1)
        return log_res if not np.isnan(log_res) else 0


def transform_log_build(year_built):
    log_built = np.log10(2020 - year_built)
    return log_built if not np.isnan(log_built) else 0

def transform_column(column_name):
    df_all_joined[column_name] = df_all_joined[column_name].progress_apply(lambda x: transform_feature(x))

for col_name in features_to_transform:
    transform_column(col_name)
df_all_joined['built_log'] = df_all_joined['yearbuilt'].apply(lambda x: transform_log_build(x))

100%|██████████| 167888/167888 [00:00<00:00, 364353.90it/s]


In [8]:
df_all_joined['built_log']

0        1.342423
1        1.724276
2        1.763428
3        1.698970
4        1.748188
5        1.579784
6        1.845098
7        1.991226
8        1.698970
9        1.602060
10       1.322219
11       1.672098
12       1.491362
13       1.602060
14       1.491362
15       1.204120
16       1.591065
17       1.591065
18       1.908485
19       1.602060
20       1.612784
21       1.623249
22       1.342423
23       2.049218
24       1.568202
25       1.880814
26       1.681241
27       1.681241
28       2.000000
29       1.724276
           ...   
90245    1.763428
90246    1.204120
90247    1.924279
90248    1.633468
90249    1.851258
90250    1.763428
90251    1.857332
90252    1.361728
90253    1.447158
90254    1.819544
90255    0.954243
90256    1.612784
90257    1.681241
90258    1.462398
90259    1.778151
90260    1.653213
90261    2.033424
90262    1.732394
90263    1.662758
90264    1.204120
90265    0.903090
90266    1.826075
90267    1.748188
90268    1.886491
90269    1

In [11]:
import sklearn.datasets
from sklearn.model_selection import train_test_split
features_to_transform.append('built_log')
X = df_all_joined[features_to_transform]
y = df_all_joined[['logerror']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
sklearn.datasets.dump_svmlight_file(X_train, y_train['logerror'], 'train_libsvm.dat')
sklearn.datasets.dump_svmlight_file(X_test, y_test['logerror'], 'test_libsvm.dat')

In [14]:
# X_train.to_csv('train_80.csv')
# X_test.to_csv('train_20.csv')
# y_train.to_csv('test_80.csv')
# y_test.to_csv('test_20.csv')

In [15]:
!aws s3 cp train_libsvm.dat s3://zillow-kaggle-awsml-2020
!aws s3 cp test_libsvm.dat s3://zillow-kaggle-awsml-2020
# !aws s3 cp test_80.csv s3://zillow-kaggle-awsml-2020
# !aws s3 cp test_20.csv s3://zillow-kaggle-awsml-2020

upload: ./train_libsvm.dat to s3://zillow-kaggle-awsml-2020/train_libsvm.dat
upload: ./test_libsvm.dat to s3://zillow-kaggle-awsml-2020/test_libsvm.dat


In [25]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

role = get_execution_role()
# bucket = 's3://zillow-kaggle-awsml-2020'
# prefix = 'kaggle-george'

In [26]:
s3_input_train = 's3://zillow-kaggle-awsml-2020/train_libsvm.dat'
s3_input_validation = 's3://zillow-kaggle-awsml-2020/test_libsvm.dat'

In [33]:
from sagemaker.amazon.amazon_estimator import get_image_uri
sess = sagemaker.Session()
container = get_image_uri('us-east-1', 'xgboost', '0.90-1')
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://zillow-kaggle-awsml-2020/output',
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1,
                        objective='reg:squarederror',
                        eval_metric='mae',
                        num_round=2000)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2020-01-29 23:55:02 Starting - Starting the training job...
2020-01-29 23:55:04 Starting - Launching requested ML instances......
2020-01-29 23:56:13 Starting - Preparing the instances for training......
2020-01-29 23:57:20 Downloading - Downloading input data...
2020-01-29 23:57:37 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value mae to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
[23:58:11] 134310x11 matrix with 1030553 entries loaded from /opt/ml/input/data/train
[23:58:11] 33578x11 matrix with 258003 entries loaded from /opt/ml/input/data/validation
INFO:root:Single

[226]#011train-mae:0.068461#011validation-mae:0.069442
[227]#011train-mae:0.068456#011validation-mae:0.069447
[228]#011train-mae:0.068453#011validation-mae:0.069447
[229]#011train-mae:0.068451#011validation-mae:0.06945
[230]#011train-mae:0.068451#011validation-mae:0.069459
[231]#011train-mae:0.068448#011validation-mae:0.069458
[232]#011train-mae:0.068443#011validation-mae:0.069461
[233]#011train-mae:0.068442#011validation-mae:0.069461
[234]#011train-mae:0.068441#011validation-mae:0.069463
[235]#011train-mae:0.068435#011validation-mae:0.069464
[236]#011train-mae:0.068432#011validation-mae:0.069464
[237]#011train-mae:0.068429#011validation-mae:0.069468
[238]#011train-mae:0.068426#011validation-mae:0.069472
[239]#011train-mae:0.068422#011validation-mae:0.06948
[240]#011train-mae:0.068417#011validation-mae:0.069477
[241]#011train-mae:0.068412#011validation-mae:0.069486
[242]#011train-mae:0.068406#011validation-mae:0.069489
[243]#011train-mae:0.068404#011validation-mae:0.06949
[244]#011trai

[371]#011train-mae:0.068077#011validation-mae:0.069914
[372]#011train-mae:0.068077#011validation-mae:0.069915
[373]#011train-mae:0.068075#011validation-mae:0.069916
[374]#011train-mae:0.068071#011validation-mae:0.069916
[375]#011train-mae:0.068069#011validation-mae:0.06992
[376]#011train-mae:0.068068#011validation-mae:0.069921
[377]#011train-mae:0.068067#011validation-mae:0.069926
[378]#011train-mae:0.068066#011validation-mae:0.069927
[379]#011train-mae:0.068062#011validation-mae:0.069934
[380]#011train-mae:0.068058#011validation-mae:0.069942
[381]#011train-mae:0.068057#011validation-mae:0.069942
[382]#011train-mae:0.068056#011validation-mae:0.069942
[383]#011train-mae:0.068052#011validation-mae:0.069948
[384]#011train-mae:0.068047#011validation-mae:0.069954
[385]#011train-mae:0.068047#011validation-mae:0.069961
[386]#011train-mae:0.068047#011validation-mae:0.069965
[387]#011train-mae:0.068039#011validation-mae:0.069966
[388]#011train-mae:0.068038#011validation-mae:0.069972
[389]#011tr

[512]#011train-mae:0.067811#011validation-mae:0.070367
[513]#011train-mae:0.067811#011validation-mae:0.070371
[514]#011train-mae:0.067809#011validation-mae:0.070376
[515]#011train-mae:0.067807#011validation-mae:0.070378
[516]#011train-mae:0.067804#011validation-mae:0.070378
[517]#011train-mae:0.067802#011validation-mae:0.07038
[518]#011train-mae:0.067801#011validation-mae:0.070381
[519]#011train-mae:0.067801#011validation-mae:0.070389
[520]#011train-mae:0.067798#011validation-mae:0.07039
[521]#011train-mae:0.067796#011validation-mae:0.070391
[522]#011train-mae:0.067794#011validation-mae:0.070392
[523]#011train-mae:0.067791#011validation-mae:0.070396
[524]#011train-mae:0.067789#011validation-mae:0.070397
[525]#011train-mae:0.067788#011validation-mae:0.070397
[526]#011train-mae:0.067786#011validation-mae:0.070398
[527]#011train-mae:0.067784#011validation-mae:0.070399
[528]#011train-mae:0.067781#011validation-mae:0.070403
[529]#011train-mae:0.067781#011validation-mae:0.070407
[530]#011tra

[660]#011train-mae:0.067541#011validation-mae:0.070741
[661]#011train-mae:0.06754#011validation-mae:0.070742
[662]#011train-mae:0.067535#011validation-mae:0.070743
[663]#011train-mae:0.067533#011validation-mae:0.070744
[664]#011train-mae:0.06753#011validation-mae:0.070748
[665]#011train-mae:0.067529#011validation-mae:0.070748
[666]#011train-mae:0.067527#011validation-mae:0.070748
[667]#011train-mae:0.067523#011validation-mae:0.070749
[668]#011train-mae:0.067521#011validation-mae:0.070751
[669]#011train-mae:0.067519#011validation-mae:0.070756
[670]#011train-mae:0.067518#011validation-mae:0.070757
[671]#011train-mae:0.067516#011validation-mae:0.070758
[672]#011train-mae:0.067515#011validation-mae:0.070758
[673]#011train-mae:0.067511#011validation-mae:0.070767
[674]#011train-mae:0.067509#011validation-mae:0.070769
[675]#011train-mae:0.067505#011validation-mae:0.07077
[676]#011train-mae:0.067504#011validation-mae:0.070771
[677]#011train-mae:0.067502#011validation-mae:0.070773
[678]#011trai

...

[1408]#011train-mae:0.066524#011validation-mae:0.072159
[1409]#011train-mae:0.066525#011validation-mae:0.072164
[1410]#011train-mae:0.066524#011validation-mae:0.072165
[1411]#011train-mae:0.066523#011validation-mae:0.072168
[1412]#011train-mae:0.066522#011validation-mae:0.072169
[1413]#011train-mae:0.066522#011validation-mae:0.072174
[1414]#011train-mae:0.066521#011validation-mae:0.072174
[1415]#011train-mae:0.066519#011validation-mae:0.072174
[1416]#011train-mae:0.066519#011validation-mae:0.072174
[1417]#011train-mae:0.066517#011validation-mae:0.07218
[1418]#011train-mae:0.066515#011validation-mae:0.072183
[1419]#011train-mae:0.066513#011validation-mae:0.072187
[1420]#011train-mae:0.066513#011validation-mae:0.072187
[1421]#011train-mae:0.06651#011validation-mae:0.072191
[1422]#011train-mae:0.066509#011validation-mae:0.072191
[1423]#011train-mae:0.066507#011validation-mae:0.072191
[1424]#011train-mae:0.066507#011validation-mae:0.072191
[1425]#011train-mae:0.066505#011validation-mae:0.0

[1557]#011train-mae:0.066368#011validation-mae:0.072443
[1558]#011train-mae:0.066366#011validation-mae:0.072445
[1559]#011train-mae:0.066364#011validation-mae:0.072446
[1560]#011train-mae:0.066361#011validation-mae:0.072447
[1561]#011train-mae:0.06636#011validation-mae:0.072449
[1562]#011train-mae:0.066358#011validation-mae:0.07245
[1563]#011train-mae:0.066356#011validation-mae:0.072454
[1564]#011train-mae:0.066353#011validation-mae:0.072454
[1565]#011train-mae:0.066351#011validation-mae:0.072455
[1566]#011train-mae:0.066348#011validation-mae:0.072455
[1567]#011train-mae:0.066349#011validation-mae:0.07246
[1568]#011train-mae:0.066347#011validation-mae:0.072463
[1569]#011train-mae:0.066345#011validation-mae:0.072466
[1570]#011train-mae:0.066344#011validation-mae:0.072468
[1571]#011train-mae:0.066342#011validation-mae:0.072469
[1572]#011train-mae:0.066342#011validation-mae:0.072469
[1573]#011train-mae:0.066343#011validation-mae:0.072469
[1574]#011train-mae:0.066342#011validation-mae:0.07

[1710]#011train-mae:0.066198#011validation-mae:0.072623
[1711]#011train-mae:0.066197#011validation-mae:0.072623
[1712]#011train-mae:0.066196#011validation-mae:0.072625
[1713]#011train-mae:0.066194#011validation-mae:0.072625
[1714]#011train-mae:0.066192#011validation-mae:0.072627
[1715]#011train-mae:0.066192#011validation-mae:0.072628
[1716]#011train-mae:0.066191#011validation-mae:0.072629
[1717]#011train-mae:0.066189#011validation-mae:0.07263
[1718]#011train-mae:0.066188#011validation-mae:0.07263
[1719]#011train-mae:0.066187#011validation-mae:0.072631
[1720]#011train-mae:0.066186#011validation-mae:0.072632
[1721]#011train-mae:0.066185#011validation-mae:0.072632
[1722]#011train-mae:0.066183#011validation-mae:0.072633
[1723]#011train-mae:0.066183#011validation-mae:0.072635
[1724]#011train-mae:0.066181#011validation-mae:0.072636
[1725]#011train-mae:0.066182#011validation-mae:0.07264
[1726]#011train-mae:0.06618#011validation-mae:0.072641
[1727]#011train-mae:0.066179#011validation-mae:0.072


2020-01-30 00:00:33 Uploading - Uploading generated training model[1862]#011train-mae:0.066027#011validation-mae:0.072838
[1863]#011train-mae:0.066026#011validation-mae:0.072838
[1864]#011train-mae:0.066025#011validation-mae:0.072839
[1865]#011train-mae:0.066024#011validation-mae:0.072842
[1866]#011train-mae:0.066023#011validation-mae:0.072846
[1867]#011train-mae:0.066022#011validation-mae:0.072846
[1868]#011train-mae:0.066021#011validation-mae:0.072847
[1869]#011train-mae:0.06602#011validation-mae:0.072848
[1870]#011train-mae:0.066019#011validation-mae:0.072849
[1871]#011train-mae:0.066018#011validation-mae:0.07285
[1872]#011train-mae:0.066017#011validation-mae:0.072851
[1873]#011train-mae:0.066016#011validation-mae:0.072851
[1874]#011train-mae:0.066015#011validation-mae:0.072852
[1875]#011train-mae:0.066015#011validation-mae:0.072852
[1876]#011train-mae:0.066013#011validation-mae:0.072852
[1877]#011train-mae:0.066014#011validation-mae:0.072855
[1878]#011train-mae:0.066013#011validat


2020-01-30 00:00:40 Completed - Training job completed
Training seconds: 200
Billable seconds: 200
